In [1]:
from skimage import io, color, transform
import numpy as np
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Read data set
data_dir = '/Users/kunpenghao/Desktop/Study_in_SIT/EE_628/project'
test_str = data_dir + '/data/test1/*.jpg'
train_str = data_dir + '/data/train/*.jpg'

def convert_gray(f): 
    rgb = io.imread(f)
    rgb = transform.resize(rgb, (28, 28))
    return color.rgb2gray(rgb)

test_im = io.ImageCollection(test_str, load_func = convert_gray)
train_im = io.ImageCollection(train_str, load_func = convert_gray)

train = [0 for i in range(25000)]
test = [0 for i in range(12500)]

time_start = time.time()
for i in range(len(train_im)):
    pic = train_im[i]
    train[i] = pic
for i in range(len(test_im)):
    pic = test_im[i]
    test[i] = pic
time_end = time.time()
print(time_end - time_start)

train = np.array(train)
test = np.array(test)

print(train.shape)
print(test.shape)

/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


217.99827694892883
(25000, 28, 28)
(12500, 28, 28)


In [3]:
# Make class for train_set
# x_train(20000), 0-9999 cat; 10000-19999 dog
# x_val(5000), 0-2499 cat; 2500-4999 dog
# y_train(20000) y_val(5000), cat = 1, dog = 0
cat_train = train[:10000]
cat_val = train[10000:12500]
dog_train = train[12500:22500]
dog_val = train[22500:, :, :]
x_train = np.zeros([20000, 28, 28])
x_val = np.zeros([5000, 28, 28])
x_train[:10000] = cat_train
x_train[10000:] = dog_train
x_val[:2500] = cat_val
x_val[2500:] = dog_val
y_train = np.zeros([20000, 1])
y_val = np.zeros([5000, 1])
y_train[10000:] = 1
y_val[2500:] = 1
y_train = y_train.tolist()
y_val = y_val.tolist()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)

In [4]:
x_test = test.reshape(test.shape[0], 28, 28, 1)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [6]:

number_of_classes = 2

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_val = np_utils.to_categorical(y_val, number_of_classes)

y_train[0], Y_train[0]

([0.0], array([1., 0.], dtype=float32))

In [7]:
#CNN nets
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

BatchNormalization(axis=-1)
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
BatchNormalization(axis=-1)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

BatchNormalization()
model.add(Dense(512))
model.add(Activation('relu'))
BatchNormalization()
model.add(Dropout(0.2))
model.add(Dense(2))

model.add(Activation('softmax'))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 10, 10, 64)        0         
__________

In [9]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [10]:
model.fit(x_train, Y_train,
                epochs=25,
                batch_size=512,
                shuffle=True,
                validation_data=(x_val, Y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 19s 943us/step - loss: 0.6821 - acc: 0.5465 - val_loss: 0.6471 - val_acc: 0.6270
Epoch 2/25
20000/20000 [==============================] - 17s 846us/step - loss: 0.6315 - acc: 0.6398 - val_loss: 0.6464 - val_acc: 0.6216
Epoch 3/25
20000/20000 [==============================] - 17s 863us/step - loss: 0.6116 - acc: 0.6660 - val_loss: 0.6157 - val_acc: 0.6608
Epoch 4/25
20000/20000 [==============================] - 17s 842us/step - loss: 0.5781 - acc: 0.6959 - val_loss: 0.5642 - val_acc: 0.7020
Epoch 5/25
20000/20000 [==============================] - 17s 851us/step - loss: 0.5544 - acc: 0.7153 - val_loss: 0.5659 - val_acc: 0.7010
Epoch 6/25
20000/20000 [==============================] - 17s 842us/step - loss: 0.5340 - acc: 0.7329 - val_loss: 0.5254 - val_acc: 0.7390
Epoch 7/25
20000/20000 [==============================] - 17s 837us/step - loss: 0.5106 - acc: 0.7502 - val_loss: 0.5

In [11]:
score = model.evaluate(x_val, Y_val)
print('Test accuracy: ', score[1])

5000/5000 [==============================] - 2s 362us/step
Test accuracy:  0.7786


In [12]:
predictions = model.predict_classes(x_test)

predictions = list(predictions)
pic = []
for i in range(12500):
    pic.append(i+1)

sub = pd.DataFrame({'id': pic,'label': predictions})
sub.to_csv('./output_CNN_1.csv', index=False)